# TSA Chapter 10: ARIMA + Fourier Forecast for Sunspots

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10_sunspot_forecast/TSA_ch10_sunspot_forecast.ipynb)

This notebook demonstrates:
- Forecasting yearly sunspot numbers using an ARIMA model with Fourier exogenous regressors to capture the ~11-year cycle.

In [ ]:
!pip install statsmodels matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Load sunspot data
data = sm.datasets.sunspots.load_pandas().data
data.index = pd.Index(data['YEAR'].astype(int))
data = data.loc[1900:2008]
y = data['SUNACTIVITY']

# Create Fourier features (period ~ 11 years)
n = len(y)
period = 11
t_arr = np.arange(1, n + 1)
exog = np.column_stack(
    [np.sin(2*np.pi*k*t_arr/period) for k in range(1, 4)] +
    [np.cos(2*np.pi*k*t_arr/period) for k in range(1, 4)]
)

n_train = n - 20
y_train = y.iloc[:n_train]
exog_train = exog[:n_train]
exog_test = exog[n_train:]

model = ARIMA(y_train, order=(2, 0, 2), exog=exog_train)
res = model.fit()
fcast = res.get_forecast(steps=20, exog=exog_test)
pred_mean = fcast.predicted_mean
ci = fcast.conf_int(alpha=0.05)

fig, ax = plt.subplots(figsize=(8, 3.5))
ax.plot(y.index[:n_train], y.values[:n_train], color=COLORS['blue'], lw=0.8, label='Training data')
ax.plot(y.index[n_train:], y.values[n_train:], color=COLORS['gray'], lw=0.8, label='Actual (test)')
forecast_idx = y.index[n_train:]
ax.plot(forecast_idx, pred_mean, color=COLORS['red'], lw=1.0, label='ARIMA+Fourier forecast')
ax.fill_between(forecast_idx, ci.iloc[:, 0], ci.iloc[:, 1], color=COLORS['red'], alpha=0.15, label='95% CI')
ax.set_xlabel('Year'); ax.set_ylabel('Sunspot Number')
ax.set_title('ARIMA + Fourier Forecast for Sunspot Numbers')
legend_outside(ax, ncol=2, y=-0.20)
fig.tight_layout()
save_chart(fig, 'sunspot_forecast')
plt.show()